<a href="https://colab.research.google.com/github/mpnsk/ivy_seminar/blob/main/ivy_complete_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ivy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 6.3 MB/s eta 0:00:00


In [14]:
import ivy
# import ivy.functional.frontends.torch.nn.functional as F

print('imported ivy')
ivy.set_torch_backend()
# ivy.set_tensorflow_backend()
# ivy.set_jax_backend()
# import jax
# jax.config.update('jax_enable_x64', True)
# ivy.set_default_device("cpu")

print('loading mnist')
from keras.datasets import mnist
(train_X_np, train_y_np), (test_X, test_y) = mnist.load_data()
# print('X_train: ' + str(train_X.shape))
# print('Y_train: ' + str(train_y.shape))
# print('X_test:  '  + str(test_X.shape))
# print('Y_test:  '  + str(test_y.shape))

# don't look at all samples
limit = 6400
train_X_np = train_X_np[:limit, :, :]
train_y_np = train_y_np[:limit]

print(f'{train_X_np.shape=}')

# ivy.set_default_device("cpu")
# ivy.set_tensorflow_backend()
# ivy.set_jax_backend()
# ivy.set_numpy_backend()

print('defining model')
class Net(ivy.Module):
    def __init__(self):
        self.conv1 = ivy.Conv2D(1, 32, [3, 3], 1, "VALID")
        self.conv2 = ivy.Conv2D(32, 64, [3, 3], 1, "VALID")
        self.conv1_drop = ivy.Dropout(0.25)
        self.conv2_drop = ivy.Dropout(0.5)
        self.fc1 = ivy.Linear(9216, 128)
        self.fc2 = ivy.Linear(128, 10)
        super().__init__()

    def _forward(self, x):
        x = self.conv1(x)
        x = ivy.relu(x)
        x = self.conv2(x)
        x = ivy.relu(x)
        x = ivy.max_pool2d(x, 2, 2, 'VALID')
        x = self.conv1_drop(x)
        print(f'before flatten: {x.shape=}')
        x = ivy.flatten(x, start_dim=1)
        # print(f'{x=}')
        print(f'after flatten: {x.shape=}')
        x = self.fc1(x)
        x = ivy.relu(x)
        x = self.conv2_drop(x)
        x = self.fc2(x)
        output = ivy.softmax(x)
        # output = ivy.softmax(x)
        return output


model = Net()
# my_x = ivy.empty((1, 28, 28, 1))
# my_x[:, :, :, 0] = ivy.array(train_X[0])

print('defined model')




def loss_fn(v, x, y):
    y_pred = model(x)
    # print(f'{x.get_num_dims()=}')
    # print(f'{y_pred=}')
    print(f'{x.get_num_dims()=}')
    print(f'{x.shape=}')
    print(f'{y_pred.get_num_dims()=}')
    print(f'{y_pred.shape=}')
    # print(f'{y_pred=}')
    print(f'{y.get_num_dims()=}')
    print(f'{y.shape=}')
    print(f'{y=}')
    # print(f'{y.get_num_dims()=}')
    entropy = ivy.sparse_cross_entropy(y, y_pred)
    # entropy = F.nll_loss(y_pred, y)
    print(f'{entropy.shape=}')
    aggregated_entropy = entropy.mean()
    print(f'{aggregated_entropy=}')
    # print(f'{entropy.get_num_dims()=}')
    return aggregated_entropy


# jax.numpy.set_printoptions(threshold=1000000)

print('starting to batch')
# turn training data into batches
## convert to ivy
train_X_ivy = ivy.array(train_X_np)
train_y_ivy = ivy.array(train_y_np)


## batch paramters
batch_size = 64
num_batches = len(train_X_ivy)

train_X_batches = [train_X_ivy[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]
train_y_batches = [train_y_ivy[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]
# train_y_batches.expand_dims(axis=-1)



print('starting training loop')
# dimension0 = train_X_batches.shape[0]
dimension0 = len(train_X_batches)
for idx in range(dimension0):
    print(f"{idx=}")
    # print(f"{type(train_X[idx])=}")
    # print(f"{type(train_y[idx])=}")
    x = train_X_batches[idx]
    x = ivy.expand_dims(x, axis=-1)
    y = train_y_batches[idx]
    # y = y.expand_dims(axis=-1)
    # print(f"{ivy.dtype(x)=}")
    # print(f"{ivy.dtype(y)=}")
    x = x / 255
    # x = ivy.expand_dims(x, axis=0)
    loss, grads = ivy.execute_with_gradients(lambda v: loss_fn(v, x, y), model.v)
    # loss, grad
    # Set a large threshold value
    print(f'{loss=}')
    print(f'{grads=}')
    if grads.all().cont_all_false():
        print("ERROR!")
        break
    # print(f'{grads.conv1.w=}')
    # print(f'{grads.conv1.b=}')
    # print(f'{grads.conv2.w=}')
    # print(f'{grads.conv2.b=}')
    # print(f'{grads.fc1.w=}')
    # print(f'{grads.fc1.b=}')
    # print(f'{grads.fc2.w=}')
    # print(f'{grads.fc2.b=}')
    model.v = model.v - grads * 0.1


imported ivy
loading mnist
train_X_np.shape=(6400, 28, 28)
defining model


AttributeError: ignored